# Toy Network

In [8]:
import numpy as np
import networkx as nx

from utils.layout import Layout
from utils.spheres import spheres, spherical_graph

In [9]:
# scene parameters
dim = 3
samples = 50
start_radius = 0.2
end_radius = 1.1
step_size = 0.05
radii = np.arange(start_radius, end_radius+step_size, step_size)

# point cloud parameters
dim = 3
rng = np.random.default_rng(1)
n1 = 10
n2 = 5
perturb_amount = (0.25, 0.25, 0.5)
scale = 0.4
center = (0.5, 1, 0)

# colors and opacities
node_color = (102, 232, 188, 255)     # turquoise
big_hole_edge_hue = (248, 177, 0)     # yellow
little_hole_edge_hue = (199, 56, 232) # violet
base_opacity = 0
birth_opacity = 255
death_opacity = 50
wireframe_node_color = (0, 0, 0, 0)
wireframe_edge_color = (128, 128, 128, 20)

# holes thresholds
big_hole_birth = 0.5
big_hole_death = 1
little_hole_birth = 0.3
little_hole_death = 0.4

# environment variables
folder = "VR/toy/"
prefix = "toy_"

In [10]:
# UTILITY FUNCTIONS FOR UPDATING COLORS AND POSITIONS

def update_opacities(big_hole: int, little_hole: int) -> None:
    """
    Updates opacities of the hole edges.
    """
    layout.edge_colors = np.concatenate((
        np.repeat([big_hole_edge_hue + (big_hole,)], repeats=n1, axis=0),
        np.repeat([little_hole_edge_hue + (little_hole,)], repeats=n2, axis=0),
        np.repeat([wireframe_edge_color], repeats=G.number_of_edges()-PC.number_of_edges(), axis=0),
    ))

def update_positions(radius: float) -> None:
    """
    Updates positions of wireframe nodes to given radius.
    """
    _, pos = spheres(PC, PC_pos, radius=radius, samples=samples, dim=dim)
    layout.pos = pos

In [11]:
# GENERATE POINT CLOUD

# network
C1 = spherical_graph(n1, dim=2)
C2 = spherical_graph(n2, dim=2)
nx.relabel_nodes(C2, lambda x: x+n1, copy=False)
PC = nx.compose(C1, C2)

# first circle positions
PC_pos = nx.circular_layout(C1, dim=dim)
for node in C1:
    PC_pos[node] = PC_pos[node] + perturb_amount * (0.5 - rng.random(dim))

# second circle positions flipped on y axis
pos = nx.circular_layout(C2, dim=dim, scale=scale, center=center)
for node in C2:
    PC_pos[node] = (pos[node] + perturb_amount * (0.5 - rng.random(dim)))[::-1]

# spheres with radius 0
G, pos = spheres(PC, PC_pos, radius=0, samples=samples, dim=dim)

In [12]:
# GENERATE LAYOUT

layout = Layout(G, pos)

# set node colors
layout.node_colors = np.concatenate((
    np.repeat([node_color], repeats=len(PC), axis=0),                      # point cloud nodes
    np.repeat([wireframe_node_color], repeats=(len(G) - len(PC)), axis=0), # sphere nodes
))

# set edge colors
update_opacities(base_opacity, base_opacity)

# save layout
layout.write(folder + prefix + "0")

In [13]:
# TEST DIFFERENT RADII
# CHECK samples VALUE FOR PREVIEWING

# update_positions(1)
# update_opacities(birth_opacity, birth_opacity)
# layout.preview(renderer="browser")

In [14]:
# LOOP THROUGH RADII AND SAVE SCENES

for i, r in enumerate(radii):
    update_positions(r)
    update_opacities(base_opacity, base_opacity)
    if little_hole_birth <= r < little_hole_death:
        update_opacities(base_opacity, birth_opacity)
    if r == little_hole_death:
        update_opacities(base_opacity, death_opacity)
    if big_hole_birth <= r < big_hole_death:
        update_opacities(birth_opacity, base_opacity)
    if r == big_hole_death:
        update_opacities(death_opacity, base_opacity)
    layout.write(folder + prefix + str(i+1))